# MODEL TRAINING

#### IMPORTING LIBRARIES AND ADDING BACKEND FOLDER TO PATH TO USE data_fetcher.py

In [1]:
# Import Libraries
import sys
import os

# Add backend to path
backend_path = os.path.abspath('../backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print("🚀 Starting Model Training with Upstox Data")

TensorFlow version: 2.16.1
🚀 Starting Model Training with Upstox Data


#### INITIALIZE DATA FETCHER

#### FETCH REAL MARKET DATA